╔══<i><b>Alai-DeepLearning</b></i>════════════════════════════╗
###  &nbsp;&nbsp; **✎&nbsp;&nbsp;Week 10. CNN Architectures**
# Section 1. VGG Net 구성하기

### _Objective_
1. VGGNet의 논문을 읽고 구현하면서, VGGNet 모델을 이해해 보겠습니다. <br>
  
╚═════════════════════════════════════════╝

In [0]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cv2

## Graph Visualization

In [0]:
from IPython.display import clear_output, Image, display, HTML
import numpy as np    

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))

    display(HTML(iframe))

<br><br>

# \[ Paper Implementation \]
---
---


> *VGG Network는 이미지 분류 대회인 ILSVRC 대회에서 2등을 기록한 모델입니다.*<br>
> *구조적으로 매우 간단하면서도, 높은 성능을 기록해, 이후 많은 모델들이 이를 기반으로 구성되었습니다.*<br>

![Imgur](https://i.imgur.com/l5DgFQo.png)

Reference : [Very Deep Convolutional Networks For Large-Scale Image Recognition](https://arxiv.org/pdf/1409.1556.pdf)

<br>

## 1. Placeholder 구성하기
---

![Imgur](https://i.imgur.com/HNgH2B6.png)

* Input의 크기는 (224,224,3)입니다.<br>
* 학습시킨 데이터셋의 라벨 수는 1,000개입니다.<br>
* 전처리로서는 image의 mean value를 빼주는 식으로 진행되었습니다.<br><br>
R Channel mean = 123.68,<br>
G Channel mean = 116.779,<br>
B Channel mean = 103.939<br>
<br>

In [0]:
input_shape = (224,224,3)
num_classes = 1000

graph = tf.Graph()
with graph.as_default():
    images = #fix me #
    labels = #fix me #
    
    with tf.variable_scope("preprocess"):
        vgg_mean = #fix me #
        x = #fix me #

In [0]:
show_graph(graph)

<br>

## 2. Inference Network 구성하기
---

![Imgur](https://i.imgur.com/16z2FhL.png)

* 위의 네트워크 중 D 타입으로 구현해보도록 하겠습니다.<br>
* Paper 마다 자신의 네트워크를 보다 잘 설명하기 위한 표기방법이 있습니다. VGGNet에서는<br>
`conv<receptive field size>-<number of channels>`방식으로 크기를 설명해주었습니다.

### (1) VGG Net의 세부 사항들 

Convolution Layer의 구성은 아래 논문에 서술 되어 있습니다.

![Imgur](https://i.imgur.com/EZWrJkY.png)

* stride : 1
* padding : SAME
* Activation Function : RELU

### (2) 첫번째 VGG Block 구성하기

VGG Net은 단순히 (3,3) 크기의 Convolution Layer가 Maxpool과 함께 적층되어 있는 구조입니다.<br> 
이전의 Network들은 (5,5), (11,11) 크기의 Filter들을 이용했는데, <br>
VGG Net은 (3,3)만으로 충분히 깊게 쌓는 방식으로 모델을 구성하였습니다.

In [0]:
with graph.as_default():
    with tf.variable_scope('VGGBlock-1'):
        conv = tf.layers.Conv2D(#fix me #)(#fix me#)
        conv = tf.layers.Conv2D(#fix me#)(#fix me#)
    pool = tf.layers.MaxPooling2D(#fix me#)(conv)

In [0]:
show_graph(graph)

### (3) 2~5번째 VGG Block 구성하기

![Imgur](https://i.imgur.com/uTGrtng.png)

VGG 연구자들은 filter size가 (7,7)인 Convolution Layer 한 층을 구성하는 것보다<br>
filter size가 (3,3)인 Convolution Layer 3 층을 구성하는 것이<br>
파라미터 수를 줄이면서, Non-Linearlity를 확보할 수 있었다고 말합니다.

이는 현대 많은 Neural Network가 큰 Filter를 쓰지 않고, (3,3) Filter만으로 깊게 쌓도록 <br>
하는데 결정적인 연구가 되었습니다.

In [0]:
with graph.as_default():
    with tf.variable_scope('VGGBlock-2'):
        #fix me#    
    with tf.variable_scope('VGGBlock-3'):
        #fix me#    
    with tf.variable_scope('VGGBlock-4'):
        #fix me#    
    with tf.variable_scope('VGGBlock-5'):
        #fix me#

In [0]:
show_graph(graph)

### (4) Receptive Field 계산하기

Convolution Layer을 적층하였을 때, 가장 먼저 체크해야 하는 것 중 하나가<br>
Convolution Layer의 Receptive Field 크기가 영상을 파악하기에 충분한 크기 수준으로<br>
확보되었는가입니다.

| Layer | Filter size | Stride | Receptive Field |
| ----  | ------ | ----- | ------ |
| block1-conv1 | (3,3) | (1,1) | (3,3) |
| block1-conv2 | (3,3) | (1,1) | (5,5) |
| maxpool1 | (2,2) | (2,2) | (6,6) |
| block2-conv1 | (3,3) | (1,1) | (10,10) |
| block2-conv2 | (3,3) | (1,1) | (14,14) |
| maxpool2 | (2,2) | (2,2) | (16,16) |
| block3-conv1 | (3,3) | (1,1) | (24,24) |
| block3-conv2 | (3,3) | (1,1) | (32,32) |
| block3-conv3 | (3,3) | (1,1) | (40,40) |
| maxpool3 | (2,2) | (2,2) | (44,44) |
| block4-conv1 | (3,3) | (1,1) | (60,60) |
| block4-conv2 | (3,3) | (1,1) | (76,76) |
| block4-conv3 | (3,3) | (1,1) | (92,92) |
| maxpool4 | (2,2) | (2,2) | (100,100) |
| block5-conv1 | (3,3) | (1,1) | (132,132) |
| block5-conv2 | (3,3) | (1,1) | (164,164) |
| block5-conv3 | (3,3) | (1,1) | (196,196) |
| maxpool5 | (2,2) | (2,2) | (212,212) |

마지막 Max pool layer의 1 pixel 당 (212,212)만큼의 입력값 공간을 바라보고 있습니다.<br>
영상의 크기가 (224,224)이므로 receptive Field의 크기가 충분하다고 파악할 수 있습니다.<br>

### (5) Fully Connected Layer 구성하기

![Imgur](https://i.imgur.com/jTYH9Ze.png)

논문에는 세부 구현 내용들이 곳곳히 숨겨져 있습니다.<br>
Fully Connected Layer에는 Dropout으로 Regularization을 해주어야 합니다.<br>


In [0]:
with graph.as_default():
    is_train = tf.placeholder_with_default(False,(),name='is_train')
    
    with tf.variable_scope('FC'):
        #fix me #
    logits = tf.identity(logits,name='logits')
    pred = tf.nn.softmax(logits,name='predictions')

In [0]:
show_graph(graph)

<br>

## 3. 학습에 관련된 부분들 구성하기
---


### (1) Loss Function 구성하기

![Imgur](https://i.imgur.com/C9RtVJA.png)

multinomial Logistic은 다른 말로 Softmax Function입니다. <br>
위의 문장은 Softmax Function with cross Entropy를 Loss Function으로써 사용하겠다라는<br>
말로 해석할 수 있습니다. <br><br>
추가로 이 논문에서는 L2 Normalization을 이용하는데, 이때 가중치는 $5*10^{-4}$를 이용합니다.

In [0]:
weight_decay = 5e-4

with graph.as_default():
    with tf.variable_scope("losses"):
        #fix me # add l2 loss 
    loss = tf.identity(loss, name='loss')

Instructions for updating:
Use tf.cast instead.


In [0]:
show_graph(graph)

### (2) Optimizer 구성하기

![Imgur](https://i.imgur.com/dVyX1fX.png)

VGG 모델을 학습시킬 때에는 Optimizer로서 Momentum Optimizer을 이용하였습니다.

In [0]:
momentum = 0.9

with graph.as_default():
    lr = #fix me #
    with tf.variable_scope("optimizer"):
        train_op = #fix me #

In [0]:
show_graph(graph)

#  

---

    Copyright(c) 2019 by Public AI. All rights reserved.<br>
    Writen by PAI, SangJae Kang ( rocketgrowthsj@publicai.co.kr )  last updated on 2019/05/07

---